# Sarcina de clasificare a textului

Așa cum am menționat, ne vom concentra pe o sarcină simplă de clasificare a textului bazată pe datasetul **AG_NEWS**, care presupune clasificarea titlurilor de știri în una dintre cele 4 categorii: Lume, Sport, Afaceri și Știință/Tehnologie.

## Datasetul

Acest dataset este integrat în modulul [`torchtext`](https://github.com/pytorch/text), așa că putem avea acces la el cu ușurință.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Aici, `train_dataset` și `test_dataset` conțin colecții care returnează perechi de etichetă (numărul clasei) și text, respectiv, de exemplu:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Așadar, să afișăm primele 10 titluri noi din setul nostru de date:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Deoarece seturile de date sunt iteratoare, dacă dorim să folosim datele de mai multe ori, trebuie să le convertim într-o listă:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizare

Acum trebuie să transformăm textul în **numere** care pot fi reprezentate ca tensori. Dacă dorim o reprezentare la nivel de cuvinte, trebuie să facem două lucruri:
* utilizăm un **tokenizator** pentru a împărți textul în **tokeni**
* construim un **vocabular** al acelor tokeni.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Folosind vocabularul, putem codifica cu ușurință șirul nostru tokenizat într-un set de numere:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Reprezentarea textului prin metoda Bag of Words

Deoarece cuvintele transmit semnificație, uneori putem înțelege sensul unui text doar analizând cuvintele individuale, indiferent de ordinea lor în propoziție. De exemplu, atunci când clasificăm știri, cuvinte precum *vreme*, *zăpadă* sunt probabil să indice *prognoza meteo*, în timp ce cuvinte precum *acțiuni*, *dolar* ar putea sugera *știri financiare*.

Reprezentarea vectorială **Bag of Words** (BoW) este cea mai utilizată metodă tradițională de reprezentare vectorială. Fiecare cuvânt este asociat unui index vectorial, iar elementul vectorului conține numărul de apariții ale unui cuvânt într-un document dat.

![Imagine care arată cum este reprezentată în memorie metoda bag of words.](../../../../../translated_images/ro/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Poți să te gândești la BoW și ca la o sumă a tuturor vectorilor one-hot-encoded pentru cuvintele individuale din text.

Mai jos este un exemplu despre cum să generezi o reprezentare bag of words folosind biblioteca python Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Pentru a calcula vectorul bag-of-words din reprezentarea vectorială a datasetului nostru AG_NEWS, putem folosi următoarea funcție:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Notă:** Aici folosim variabila globală `vocab_size` pentru a specifica dimensiunea implicită a vocabularului. Deoarece dimensiunea vocabularului este adesea destul de mare, putem limita dimensiunea vocabularului la cele mai frecvente cuvinte. Încercați să reduceți valoarea `vocab_size` și să rulați codul de mai jos, și observați cum afectează acuratețea. Ar trebui să vă așteptați la o scădere a acurateței, dar nu dramatică, în schimbul unei performanțe mai ridicate.


## Antrenarea clasificatorului BoW

Acum că am învățat cum să construim reprezentarea Bag-of-Words a textului nostru, să antrenăm un clasificator pe baza acesteia. Mai întâi, trebuie să convertim setul nostru de date pentru antrenare astfel încât toate reprezentările vectoriale poziționale să fie transformate în reprezentări Bag-of-Words. Acest lucru poate fi realizat prin transmiterea funcției `bowify` ca parametru `collate_fn` la `DataLoader` standard din torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Acum să definim o rețea neuronală clasificatoare simplă care conține un singur strat liniar. Dimensiunea vectorului de intrare este egală cu `vocab_size`, iar dimensiunea de ieșire corespunde numărului de clase (4). Deoarece rezolvăm o sarcină de clasificare, funcția de activare finală este `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Acum vom defini bucla standard de antrenament PyTorch. Deoarece setul nostru de date este destul de mare, pentru scopul nostru educativ vom antrena doar pentru o epocă, și uneori chiar pentru mai puțin de o epocă (specificarea parametrului `epoch_size` ne permite să limităm antrenamentul). De asemenea, vom raporta acuratețea acumulată a antrenamentului în timpul procesului; frecvența raportării este specificată folosind parametrul `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrame, TriGrame și N-Grame

O limitare a abordării sacului de cuvinte este că unele cuvinte fac parte din expresii formate din mai multe cuvinte. De exemplu, cuvântul „hot dog” are un sens complet diferit față de cuvintele „hot” și „dog” în alte contexte. Dacă reprezentăm întotdeauna cuvintele „hot” și „dog” prin aceleași vectori, acest lucru poate deruta modelul nostru.

Pentru a rezolva această problemă, **reprezentările N-gram** sunt adesea utilizate în metodele de clasificare a documentelor, unde frecvența fiecărui cuvânt, bi-cuvânt sau tri-cuvânt este o caracteristică utilă pentru antrenarea clasificatorilor. În reprezentarea bigramelor, de exemplu, vom adăuga toate perechile de cuvinte în vocabular, pe lângă cuvintele originale.

Mai jos este un exemplu despre cum să generăm o reprezentare de tip sac de cuvinte cu bigrame folosind Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Principalul dezavantaj al abordării N-gram este că dimensiunea vocabularului începe să crească extrem de rapid. În practică, trebuie să combinăm reprezentarea N-gram cu unele tehnici de reducere a dimensionalității, cum ar fi *embeddings*, pe care le vom discuta în unitatea următoare.

Pentru a utiliza reprezentarea N-gram în setul nostru de date **AG News**, trebuie să construim un vocabular special de tip ngram:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Am putea folosi același cod de mai sus pentru a antrena clasificatorul, totuși, acest lucru ar fi foarte ineficient din punct de vedere al memoriei. În unitatea următoare, vom antrena un clasificator bigram folosind embeddings.

> **Notă:** Poți păstra doar acele ngrame care apar în text de mai multe ori decât numărul specificat. Acest lucru va asigura că bigramele rare vor fi omise și va reduce semnificativ dimensiunea vocabularului. Pentru a face acest lucru, setează parametrul `min_freq` la o valoare mai mare și observă cum se schimbă lungimea vocabularului.


## Frecvența Termenilor și Frecvența Inversă a Documentelor (TF-IDF)

În reprezentarea BoW, aparițiile cuvintelor sunt ponderate uniform, indiferent de cuvântul în sine. Totuși, este evident că anumite cuvinte frecvente, precum *un*, *în*, etc., sunt mult mai puțin importante pentru clasificare decât termenii specializați. De fapt, în majoritatea sarcinilor NLP, unele cuvinte sunt mai relevante decât altele.

**TF-IDF** este abrevierea pentru **frecvența termenilor – frecvența inversă a documentelor**. Este o variație a modelului bag of words, unde, în loc de o valoare binară 0/1 care indică apariția unui cuvânt într-un document, se folosește o valoare în virgulă mobilă, care este legată de frecvența apariției cuvântului în corpus.

Mai formal, greutatea $w_{ij}$ a unui cuvânt $i$ în documentul $j$ este definită astfel:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
unde
* $tf_{ij}$ este numărul de apariții ale lui $i$ în $j$, adică valoarea BoW pe care am văzut-o anterior
* $N$ este numărul de documente din colecție
* $df_i$ este numărul de documente care conțin cuvântul $i$ în întreaga colecție

Valoarea TF-IDF $w_{ij}$ crește proporțional cu numărul de apariții ale unui cuvânt într-un document și este ajustată în funcție de numărul de documente din corpus care conțin acel cuvânt, ceea ce ajută la compensarea faptului că unele cuvinte apar mai frecvent decât altele. De exemplu, dacă un cuvânt apare în *fiecare* document din colecție, $df_i=N$, și $w_{ij}=0$, iar acești termeni vor fi complet ignorați.

Poți crea cu ușurință o vectorizare TF-IDF a textului folosind Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Concluzie

Totuși, deși reprezentările TF-IDF oferă o pondere bazată pe frecvența diferitelor cuvinte, ele nu pot reprezenta sensul sau ordinea. Așa cum a spus faimosul lingvist J. R. Firth în 1935: „Sensul complet al unui cuvânt este întotdeauna contextual, iar niciun studiu al sensului în afara contextului nu poate fi luat în serios.”. Vom învăța mai târziu în curs cum să capturăm informațiile contextuale din text folosind modelarea limbajului.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
